# Segmenting and Clustering Neighborhoods in Toronto
## Part 1: Scraping and Cleaning Data for a great Data Frame

### Starting with install and import of all relevant packages and libraries

In [5]:
import pandas as pd
import numpy as np
import requests

# import and install Beutiful Soup
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

# import and install Nomniatim Geocoder
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge geocoder --yes
from geopy.geocoders import Nominatim 

# import and install JSON Normalizer
from pandas.io.json import json_normalize

# import and install folium map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

# import and install k-means 
from sklearn.cluster import KMeans

# import and install Matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### Start Data Scraping

In [6]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
PostalCodesOfToronto = BeautifulSoup(r.text, 'html.parser')

In [7]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

# loop through to find postcode, borough, neighborhood 
content = PostalCodesOfToronto.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n').replace(']','')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean the dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
print("Shape: ", df.shape)

Shape:  (103, 3)


## Part 2: Latitude and Longitude Aggregation for the basic dataframe

In [9]:
import pandas as pd

In [10]:
df_geo_coor = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_toronto = pd.merge(df, df_geo_coor, how='left', left_on = 'Postalcode', right_on = 'Postal Code')
# remove the "Postal Code" column
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
map_toronto

In [19]:
# adding map markers
for lat, lng, borough, neighborhood in zip(
        df_toronto['Latitude'], 
        df_toronto['Longitude'], 
        df_toronto['Borough'], 
        df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto



## Part 3: Exploring and Clustering all neighborhoods containing 'Toronto'

In [21]:
# all Boroughs containing 'Toronto' will be defined as 'TorontoBoroughs'
df_TorontoBoroughs = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_TorontoBoroughs.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [23]:
map_TorontoBoroughs = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, lng, borough, neighborhood in zip(
        df_TorontoBoroughs['Latitude'], 
        df_TorontoBoroughs['Longitude'], 
        df_TorontoBoroughs['Borough'], 
        df_TorontoBoroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TorontoBoroughs)  

map_TorontoBoroughs

### First Neighborhood Exploration

In [25]:
neighborhood_name = df_TorontoBoroughs.loc[0, 'Neighborhood']
print(f"Toronto's first neighborhood's name is '{neighborhood_name}'.")

Toronto's first neighborhood's name is 'The Beaches'.


In [27]:
# So let's look up 'The Beaches' Latitude and Longitude:
neighborhood_latitude = df_TorontoBoroughs.loc[0, 'Latitude'] 
neighborhood_longitude = df_TorontoBoroughs.loc[0, 'Longitude'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec1b58777af030029021f8f'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### All Neighborhood Exploration (within the Boroughs containing 'Toronto')

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        # deleted for security reasons

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
toronto_venues = getNearbyVenues(names=df_TorontoBoroughs['Neighborhood'],
                                   latitudes=df_TorontoBoroughs['Latitude'],
                                   longitudes=df_TorontoBoroughs['Longitude']
                                )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

In [77]:
toronto_venues.head(100)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,The Beaches,43.676357,-79.293031,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,The Beaches,43.676357,-79.293031,Indigo,43.653515,-79.380696,Bookstore
3,The Beaches,43.676357,-79.293031,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop
4,The Beaches,43.676357,-79.293031,Textile Museum of Canada,43.654396,-79.386500,Art Museum
...,...,...,...,...,...,...,...
95,"The Danforth West, Riverdale",43.679557,-79.352188,Richmond Station,43.651569,-79.379266,American Restaurant
96,"The Danforth West, Riverdale",43.679557,-79.352188,Solei Tanning Salon,43.654734,-79.380248,Tanning Salon
97,"The Danforth West, Riverdale",43.679557,-79.352188,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant
98,"The Danforth West, Riverdale",43.679557,-79.352188,Yonge-Dundas Square,43.656054,-79.380495,Plaza


In [40]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,71,71,71,71,71,71
"Brockton, Parkdale Village, Exhibition Place",71,71,71,71,71,71
Business reply mail Processing Centre,71,71,71,71,71,71
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",71,71,71,71,71,71
Central Bay Street,71,71,71,71,71,71
Christie,71,71,71,71,71,71
Church and Wellesley,71,71,71,71,71,71
"Commerce Court, Victoria Hotel",71,71,71,71,71,71
Davisville,71,71,71,71,71,71


In [42]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 52 uniques categories.


In [43]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Women's Store,American Restaurant,Art Museum,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Clothing Store,Cocktail Bar,...,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Women's Store,American Restaurant,Art Museum,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Clothing Store,...,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,Berczy Park,0.014085,0.028169,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.084507,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.028169,0.028169,0.014085,0.014085,0.014085
1,"Brockton, Parkdale Village, Exhibition Place",0.014085,0.028169,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.084507,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.028169,0.028169,0.014085,0.014085,0.014085
2,Business reply mail Processing Centre,0.014085,0.028169,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.084507,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.028169,0.028169,0.014085,0.014085,0.014085
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.014085,0.028169,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.084507,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.028169,0.028169,0.014085,0.014085,0.014085
4,Central Bay Street,0.014085,0.028169,0.014085,0.014085,0.014085,0.014085,0.014085,0.014085,0.084507,...,0.014085,0.014085,0.014085,0.014085,0.014085,0.028169,0.028169,0.014085,0.014085,0.014085


In [50]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2          Restaurant  0.04
3      Cosmetics Shop  0.03
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2          Restaurant  0.04
3      Cosmetics Shop  0.03
4  Seafood Restaurant  0.03


----Business reply mail Processing Centre----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2          Restaurant  0.04
3      Cosmetics Shop  0.03
4  Seafood Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0      Clothing Store  0.08
1         Coffee Shop  0.07
2          Restaurant  0.04
3      Cosmetics Shop  0.03
4  Seafood Restaurant  0.03


----Central Bay Street----
                venue  freq
0      Clothing Store  0.08
1  

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
1,"Brockton, Parkdale Village, Exhibition Place",Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
2,Business reply mail Processing Centre,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
3,"CN Tower, King and Spadina, Railway Lands, Har...",Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
4,Central Bay Street,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater


In [58]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [67]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ValueError: cannot insert Cluster Labels, already exists

In [69]:
toronto_denc_merged = df_TorontoBoroughs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_denc_merged = toronto_denc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_denc_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater


In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster Examination

In [71]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 0, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
1,East Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
2,East Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
3,East Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
4,Central Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
5,Central Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
6,Central Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
7,Central Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
8,Central Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater
9,Central Toronto,0,Clothing Store,Coffee Shop,Restaurant,Diner,Cosmetics Shop,Plaza,Seafood Restaurant,Hotel,American Restaurant,Theater


In [72]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 1, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [73]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 2, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [74]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 3, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [75]:
toronto_denc_merged.loc[toronto_denc_merged['Cluster Labels'] == 4, toronto_denc_merged.columns[[1] + list(range(5, toronto_denc_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
